In [1]:
import requests
import pandas as pd
import json
from pandas import json_normalize
import urllib

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

In [3]:
q = input("Enter Location:")
q = q.replace(" ",'%20')
load = "https://www.oyorooms.com/api/pwa/autocompletenew?query="+q
print(load)

Enter Location:varkala
https://www.oyorooms.com/api/pwa/autocompletenew?query=varkala


In [4]:
search = requests.get(load, headers = headers)

In [5]:
search_data = json.loads(search.text)

In [6]:
#Evaluating JSON

In [7]:
search_df = json_normalize(search_data['responseObject'])

In [8]:
print(search_df[['displayName','name','centerPoint.lat','centerPoint.lng','city.id','city.name','state.id','state.name']])

                                  displayName                        name  \
0                      Varkala, Kerala, India                     Varkala   
1  Trivandrum Railway Station, Varkala, India  Trivandrum Railway Station   
2              Varkala Beach, Varkala, Kerala               Varkala Beach   
3       Varkala cliff, Varkala, Kerala 695141               Varkala cliff   
4                North Cliff, Varkala, Kerala                 North Cliff   
5          Black Beach Resort, Varkala, India          Black Beach Resort   

   centerPoint.lat  centerPoint.lng  city.id   city.name  state.id state.name  
0         8.737868        76.716336      132     Varkala        13     Kerala  
1         8.486948        76.952580       53  Trivandrum        13     Kerala  
2         8.735556        76.703290      132     Varkala        13     Kerala  
3         8.735410        76.704057      132     Varkala        13     Kerala  
4         8.737571        76.701540      132     Varkala    

In [9]:
#Making URL for location page

In [10]:
search_data['responseObject'][0]['name']

'Varkala'

In [11]:
search_data['responseObject'][0]['city']['name']

'Varkala'

In [12]:
loc = search_data['responseObject'][0]['name']+", "+search_data['responseObject'][0]['city']['name']+", "+search_data['responseObject'][0]['state']['name']

In [13]:
lat = search_data['responseObject'][0]['centerPoint']['lat']
log = search_data['responseObject'][0]['centerPoint']['lng']

In [14]:
checkin = '12/03/2022'
checkout = '13/03/2022'

In [15]:
pagepayload =  {'location' : loc, 'latitude' : lat, 'longitude' : log, 'searchType' : 'locality', 'coupon' : '', 'checkin' : checkin, 'checkout' : checkout, 'roomConfig[]' : 2, 'country' : 'india', 'guests' : 2, 'rooms' : 1}

In [16]:
pagepayload

{'location': 'Varkala, Varkala, Kerala',
 'latitude': 8.7378685,
 'longitude': 76.71633589999999,
 'searchType': 'locality',
 'coupon': '',
 'checkin': '12/03/2022',
 'checkout': '13/03/2022',
 'roomConfig[]': 2,
 'country': 'india',
 'guests': 2,
 'rooms': 1}

In [17]:
urllib.parse.urlencode(pagepayload) #default quate_plus encoding, so we need to change it to quote encoding

'location=Varkala%2C+Varkala%2C+Kerala&latitude=8.7378685&longitude=76.71633589999999&searchType=locality&coupon=&checkin=12%2F03%2F2022&checkout=13%2F03%2F2022&roomConfig%5B%5D=2&country=india&guests=2&rooms=1'

In [18]:
url_encode1 = urllib.parse.urlencode(pagepayload, quote_via=urllib.parse.quote) #quote encoded
print(url_encode1)

location=Varkala%2C%20Varkala%2C%20Kerala&latitude=8.7378685&longitude=76.71633589999999&searchType=locality&coupon=&checkin=12%2F03%2F2022&checkout=13%2F03%2F2022&roomConfig%5B%5D=2&country=india&guests=2&rooms=1


In [19]:
#encoding twice for api request (oyo api needed get request with dual utf-8 encoding
#adding search prefix for get before utf-8 encoding
url_encode2 = urllib.parse.quote('/search?'+url_encode1,safe='') 
print(url_encode2)

%2Fsearch%3Flocation%3DVarkala%252C%2520Varkala%252C%2520Kerala%26latitude%3D8.7378685%26longitude%3D76.71633589999999%26searchType%3Dlocality%26coupon%3D%26checkin%3D12%252F03%252F2022%26checkout%3D13%252F03%252F2022%26roomConfig%255B%255D%3D2%26country%3Dindia%26guests%3D2%26rooms%3D1


In [20]:
#Final url of listing page
urlfinal = 'https://www.oyorooms.com/api/pwa/getListingPage?url='+url_encode2
print(urlfinal)

https://www.oyorooms.com/api/pwa/getListingPage?url=%2Fsearch%3Flocation%3DVarkala%252C%2520Varkala%252C%2520Kerala%26latitude%3D8.7378685%26longitude%3D76.71633589999999%26searchType%3Dlocality%26coupon%3D%26checkin%3D12%252F03%252F2022%26checkout%3D13%252F03%252F2022%26roomConfig%255B%255D%3D2%26country%3Dindia%26guests%3D2%26rooms%3D1


In [21]:
listing = requests.get(urlfinal, headers = headers)

In [22]:
print(listing.text)

{"searchData":{"request_id":"2edd5304-9e3e-40d2-9bc6-6c1b63455373||gr_ChIJh4aphCXvBTsRL7i54jsIUWI","uuid":"2edd5304-9e3e-40d2-9bc6-6c1b63455373","count":50,"room_limit_info":{"pah_message":"Pay at hotel is not available when booking more than 3 rooms","pah_max_rooms":3,"allowed_rooms":4},"total_count_without_applicable_filters":50,"urgency_info":{"text":"Please check the travel advisory issued by the concerned state government/local authorities before booking, as some places may have COVID-19 related travel/lodging restrictions.","icon":"","type":"advisory"},"slasher_percentage":15,"basic_amenities_label":"Select amenities below:","ratings_enabled":true,"formatted_checkin_time":"12:00 PM","formatted_checkout_time":"11:00 AM","distance_multiplier":1,"new_applicable_filters":[{"server_key_name":"oyo_wizard","is_guided":0,"priority":8,"display_name":"OYO WIZARD","guided_display_name":"Oyo Wizard Filter","expandable_label":"OYO WIZARD","items":[{"id":"oyo_wizard_filter","name":"Show Only 

In [23]:
#Evaluating Listing Page Data

In [24]:
listing_data = json.loads(listing.text) #response text to json

In [25]:
print(json.dumps(listing_data, indent = 3)) #display json

{
   "searchData": {
      "request_id": "2edd5304-9e3e-40d2-9bc6-6c1b63455373||gr_ChIJh4aphCXvBTsRL7i54jsIUWI",
      "uuid": "2edd5304-9e3e-40d2-9bc6-6c1b63455373",
      "count": 50,
      "room_limit_info": {
         "pah_message": "Pay at hotel is not available when booking more than 3 rooms",
         "pah_max_rooms": 3,
         "allowed_rooms": 4
      },
      "total_count_without_applicable_filters": 50,
      "urgency_info": {
         "text": "Please check the travel advisory issued by the concerned state government/local authorities before booking, as some places may have COVID-19 related travel/lodging restrictions.",
         "icon": "\ue901",
         "type": "advisory"
      },
      "slasher_percentage": 15,
      "basic_amenities_label": "Select amenities below:",
      "ratings_enabled": true,
      "formatted_checkin_time": "12:00 PM",
      "formatted_checkout_time": "11:00 AM",
      "distance_multiplier": 1,
      "new_applicable_filters": [
         {
        

In [26]:
#Columns in each hotel

In [27]:
hotels_list = listing_data['searchData']['hotels']

In [28]:
listing_df = pd.DataFrame(hotels_list)

In [29]:
listing_df.keys()

Index(['shortlisted', 'show_original_name', 'urgency_info_list', 'distance',
       'longitude', 'latitude', 'currency_id', 'selected_category_id',
       'country_id', 'id', 'oyo_id', 'street', 'map_link', 'geo_location',
       'category', 'display_category', 'status', 'alternate_name', 'name',
       'address', 'short_address', 'city', 'country_name', 'hotel_type',
       'hotel_name_without_category', 'best_image', 'currency_symbol',
       'currency_code', 'distance_unit', 'external_booking_url',
       'urgency_info', 'ratings', 'pricing_details', 'amenities',
       'category_wise_media', 'category_wise_pricing',
       'room_categories_with_data', 'cancellation_policies', 'available_rooms',
       'hotel_images', 'hotel_badge', 'restrictions', 'pricing_info', 'images',
       'pricing', 'reduced_room_pricing', 'category_wise_display_pricing',
       'fixed_pricing', 'mrcData', 'rating_text', 'oyo_wizard'],
      dtype='object')

In [30]:
listing_df['ratings']

0     {'count': 255, 'value': 3.9, 'agg_type': 'deci...
1                                 {'social_rating': {}}
2     {'count': 52, 'value': 3.6, 'agg_type': 'decim...
3     {'count': 35, 'value': 2.7, 'agg_type': 'decim...
4     {'count': 107, 'value': 3.2, 'agg_type': 'deci...
5     {'count': 1007, 'value': 4.6, 'agg_type': 'dec...
6     {'count': 20, 'value': 3.1, 'agg_type': 'decim...
7     {'count': 35, 'value': 3.3, 'agg_type': 'decim...
8     {'count': 27, 'value': 3.9, 'agg_type': 'decim...
9     {'count': 393, 'value': 4, 'agg_type': 'decima...
10    {'count': 305, 'value': 3.8, 'agg_type': 'deci...
11    {'count': 321, 'value': 4, 'agg_type': 'decima...
12    {'count': 357, 'value': 4, 'agg_type': 'decima...
13    {'count': 1, 'value': 4, 'agg_type': 'decimal'...
14    {'count': 32, 'value': 3.1, 'agg_type': 'decim...
15    {'count': 50, 'value': 4.4, 'agg_type': 'decim...
16    {'count': 10, 'value': 4.6, 'agg_type': 'decim...
17    {'count': 222, 'value': 3.7, 'agg_type': '

In [31]:
#json to dataframe

In [32]:
hotel_df = json_normalize(hotels_list)

In [33]:
len(hotel_df)

20

In [34]:
hotel_df.keys()

Index(['shortlisted', 'show_original_name', 'urgency_info_list', 'distance',
       'longitude', 'latitude', 'currency_id', 'selected_category_id',
       'country_id', 'id',
       ...
       'category_wise_display_pricing.10.discount_percentage',
       'category_wise_display_pricing.10.wizard_discount_percentage',
       'category_wise_display_pricing.10.wizard_discount_amount',
       'category_wise_display_pricing.10.coupon_discount.coupon_code',
       'category_wise_display_pricing.10.coupon_discount.discount_amount',
       'category_wise_display_pricing.10.coupon_discount.discount_percentage',
       'category_wise_display_pricing.10.coupon_discount.discount_info.inventory_offers_left',
       'urgency_info.ratingCount', 'pricing_details.services.meals',
       'ratings.tag'],
      dtype='object', length=311)

In [35]:
hotel_df[['distance', 'geo_location', 'category', 'name','address', 'city','hotel_type','best_image', 'currency_symbol', 'distance_unit']].head(1)

,distance,geo_location,category,name,address,city,hotel_type,best_image,currency_symbol,distance_unit
0,0.9851,"8.734014,76.724406",SPOT ON,SPOT ON 73862 Jaya Tourist Home,"Jaya Tourist Home, Temple Rd, near South India...",Varkala,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,₹,km


In [36]:
#hotel URL
hotel_url = 'https://www.oyorooms.com/' + hotel_df['id'] + '/?rooms=1&guests=2&rooms_config=1-2_0'
print(hotel_url)

0     https://www.oyorooms.com/103848/?rooms=1&guest...
1     https://www.oyorooms.com/173906/?rooms=1&guest...
2     https://www.oyorooms.com/104197/?rooms=1&guest...
3     https://www.oyorooms.com/103286/?rooms=1&guest...
4     https://www.oyorooms.com/107100/?rooms=1&guest...
5     https://www.oyorooms.com/42911/?rooms=1&guests...
6     https://www.oyorooms.com/55572/?rooms=1&guests...
7     https://www.oyorooms.com/162962/?rooms=1&guest...
8     https://www.oyorooms.com/161573/?rooms=1&guest...
9     https://www.oyorooms.com/81135/?rooms=1&guests...
10    https://www.oyorooms.com/111242/?rooms=1&guest...
11    https://www.oyorooms.com/110365/?rooms=1&guest...
12    https://www.oyorooms.com/110373/?rooms=1&guest...
13    https://www.oyorooms.com/15986/?rooms=1&guests...
14    https://www.oyorooms.com/112598/?rooms=1&guest...
15    https://www.oyorooms.com/167612/?rooms=1&guest...
16    https://www.oyorooms.com/168351/?rooms=1&guest...
17    https://www.oyorooms.com/115587/?rooms=1&g

In [37]:
#hotel ratings and count
hotel_df['ratings.value'].fillna(0).astype(str) + " (" + hotel_df['ratings.count'].fillna(0).astype(int).astype(str) + ")" 

0      3.9 (255)
1        0.0 (0)
2       3.6 (52)
3       2.7 (35)
4      3.2 (107)
5     4.6 (1007)
6       3.1 (20)
7       3.3 (35)
8       3.9 (27)
9      4.0 (393)
10     3.8 (305)
11     4.0 (321)
12     4.0 (357)
13       4.0 (1)
14      3.1 (32)
15      4.4 (50)
16      4.6 (10)
17     3.7 (222)
18       0.0 (0)
19       5.0 (1)
dtype: object

In [38]:
hotel_df['mrcData'][0]

[{'id': '272949',
  'name': 'SPOT ON',
  'bestImage': 'https://images.oyoroomscdn.com/uploads/hotel_image/103848/85698fe458477382.jpg',
  'finalPrice': '1285',
  'slasherPrice': '2447',
  'isSoldOut': False},
 {'id': '272950',
  'name': 'SPOT ON NON-AC',
  'bestImage': 'https://images.oyoroomscdn.com/uploads/hotel_image/103848/b6c4560984d3a355.jpg',
  'finalPrice': '899',
  'slasherPrice': '1948',
  'isSoldOut': False},
 {'id': '273162',
  'name': 'SPOT ON SAVER',
  'bestImage': 'https://images.oyoroomscdn.com/uploads/hotel_image/103848/c10fc26cec82512c.jpg',
  'finalPrice': '899',
  'slasherPrice': '2344',
  'isSoldOut': False}]

In [39]:
json_normalize(hotel_df['mrcData'].apply(lambda x: x[0])) # selected 1st index type of oyo rooms

,id,name,bestImage,finalPrice,slasherPrice,isSoldOut
0,272949,SPOT ON,https://images.oyoroomscdn.com/uploads/hotel_i...,1285,2447,False
1,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,1437,2737,False
2,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,2702,5147,False
3,272950,SPOT ON NON-AC,https://images.oyoroomscdn.com/uploads/hotel_i...,899,2441,False
4,273163,SPOT ON NON-AC SAVER,https://images.oyoroomscdn.com/uploads/hotel_i...,883,1682,False
5,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,3494,6655,False
6,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,2605,4962,False
7,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,1149,3488,False
8,1,Classic (2X),https://images.oyoroomscdn.com/uploads/hotel_i...,999,3111,False
9,272950,SPOT ON NON-AC,https://images.oyoroomscdn.com/uploads/hotel_i...,1029,1959,False


In [40]:
hotel_df['amenities'].apply(lambda x:x[0])

0                    {'13': 'AC'}
1             {'53': 'Free Wifi'}
2     {'129': 'Parking Facility'}
3                    {'39': 'TV'}
4     {'129': 'Parking Facility'}
5     {'129': 'Parking Facility'}
6     {'129': 'Parking Facility'}
7           {'77': 'Living Room'}
8              {'9': 'Reception'}
9              {'9': 'Reception'}
10                   {'13': 'AC'}
11    {'129': 'Parking Facility'}
12    {'129': 'Parking Facility'}
13    {'129': 'Parking Facility'}
14              {'173': 'Window'}
15    {'129': 'Parking Facility'}
16    {'129': 'Parking Facility'}
17             {'9': 'Reception'}
18            {'53': 'Free Wifi'}
19            {'53': 'Free Wifi'}
Name: amenities, dtype: object

In [41]:
hotel_df['pricing_info']

0     [{'prices': [1352, 1641, 20], 'average_price_s...
1     [{'prices': [1666, 1916, 334], 'average_price_...
2     [{'prices': [2932, 3603, 804], 'average_price_...
3     [{'prices': [2192, 2648, 559], 'average_price_...
4     [{'prices': [1352, 1641, 222], 'average_price_...
5     [{'prices': [3977, 4659, 1175], 'average_price...
6     [{'prices': [2949, 3474, 667], 'average_price_...
7     [{'prices': [2124, 2442, 425], 'average_price_...
8     [{'prices': [1894, 2178, 379], 'average_price_...
9     [{'prices': [1184, 1372, 289], 'average_price_...
10    [{'prices': [2053, 2361, 410], 'average_price_...
11    [{'prices': [2460, 2823, 485], 'average_price_...
12    [{'prices': [1643, 1890, 329], 'average_price_...
13    [{'prices': [2341, 2692, 468], 'average_price_...
14    [{'prices': [855, 983, 171], 'average_price_si...
15    [{'prices': [1838, 2114, 367], 'average_price_...
16    [{'prices': [2007, 2308, 401], 'average_price_...
17    [{'prices': [1729, 1989, 346], 'average_pr

In [ ]:
#Search Characters

In [ ]:
import re

In [ ]:
for i in re.finditer("1492",listing.text):
    a,b = i.span()[0],i.span()[1]
    print(listing.text[a-50:b+50])

In [ ]:
hotel_df['available_rooms'].apply(lambda x: x[0])

In [42]:
#Final Table

In [71]:
excel_df = pd.DataFrame()

In [72]:
excel_df['Hotel Name'] = hotel_df['name']
excel_df['Distance'] = hotel_df['distance']
excel_df['Type'] = hotel_df['hotel_type']
excel_df['Image URL'] = hotel_df['best_image']
excel_df['Price'] = json_normalize(hotel_df['mrcData'].apply(lambda x: x[0]))['finalPrice']
excel_df['Available Rooms'] = hotel_df['available_rooms'].apply(lambda x: x[0])
excel_df['Rating'] = hotel_df['ratings.value'].fillna(0).astype(str) + " (" + hotel_df['ratings.count'].fillna(0).astype(int).astype(str) + ")" 
excel_df['Address'] = hotel_df['address']
excel_df['City'] = hotel_df['city']
excel_df['Hotel link'] = 'https://www.oyorooms.com/' + hotel_df['id'] + '/?rooms=1&guests=2&rooms_config=1-2_0'
excel_df['OYO ID'] = hotel_df['oyo_id']
excel_df['Geo Location'] = hotel_df['geo_location']
excel_df['Category'] = hotel_df['category']

In [ ]:
#excel_df[[OYO ID', 'Geo Location']] = 'background-color: yellow'

In [73]:
excel_style_df = excel_df.style.set_properties(**{'background-color': 'yellow'}, subset=['OYO ID', 'Geo Location'])

In [74]:
type(excel_style_df)

pandas.io.formats.style.Styler

In [76]:
excel_style_df

,Hotel Name,Distance,Type,Image URL,Price,Available Rooms,Rating,Address,City,Hotel link,OYO ID,Geo Location,Category
0,SPOT ON 73862 Jaya Tourist Home,0.985100,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/103848/c10fc26cec82512c.jpg,1285,2,3.9 (255),"Jaya Tourist Home, Temple Rd, near South Indian Bank, Maithanam, Varkala, Varkala",Varkala,https://www.oyorooms.com/103848/?rooms=1&guests=2&rooms_config=1-2_0,VKL021,"8.734014,76.724406",SPOT ON
1,POP 87198 Poornachandra,15.642800,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/173906/7b3d5af0d3048ad2.jpg,1437,3,0.0 (0),"RRCJ,5RQ, Thiruvananthapuram, Kerala 695604, India, Kollam, Kollam",Kollam,https://www.oyorooms.com/173906/?rooms=1&guests=2&rooms_config=1-2_0,KOM069,"8.8211399,76.8310178",POP
2,CAPITAL O74172 Four Squares Inn,20.037400,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/104197/6e4be5b95b62f01b.jpg,2702,27,3.6 (52),"Vamanapuram,Thiruvananthapuram, Trivandrum",Trivandrum,https://www.oyorooms.com/104197/?rooms=1&guests=2&rooms_config=1-2_0,TVM293,"8.725963,76.898202",Capital O
3,SPOT ON 73501 San Tower Hotel,20.036200,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/103286/2eb4065ab0ed80bc.jpg,899,2,2.7 (35),"Karette Junction, SH 1, Karette, Trivandrum",Trivandrum,https://www.oyorooms.com/103286/?rooms=1&guests=2&rooms_config=1-2_0,TVM289,"8.7317859957749,76.8984845280647",SPOT ON
4,SPOT ON 76293 J P Tourist Home,21.607600,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/107100/266fe888d77f3ae5.jpg,883,33,3.2 (107),"Paikkada Road, Kallupalam, Chinnakkada, Kollam, Kollam",Kollam,https://www.oyorooms.com/107100/?rooms=1&guests=2&rooms_config=1-2_0,KOM060,"8.88424897434428,76.5870909579098",SPOT ON
5,Capital O 19742 Hotel Dona Castle,22.440900,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/42911/a695a0653f668ac7.jpg,3494,12,4.6 (1007),"Lakshminada Main Road, Kottamukku, High School Junction-Kottamukk Rd, Vidya Nagar, Kollam",Kollam,https://www.oyorooms.com/42911/?rooms=1&guests=2&rooms_config=1-2_0,KOM013,"8.88758520155754,76.5794709697366",Capital O
6,OYO 37062 Lee Frank,25.002800,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/55572/69dc5a84c9c03e25.jpg,2605,30,3.1 (20),"Menamkulam, Kazhakkoottam, Trivandrum",Trivandrum,https://www.oyorooms.com/55572/?rooms=1&guests=2&rooms_config=1-2_0,TVM185,"8.56328564739182,76.8595690280199",OYO Rooms
7,OYO 84410 Eden Suites,25.060200,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/162962/ed851e40db90e3e7.jpg,1149,15,3.3 (35),"Kazhakkoottam, Kaniyapuram, Thiruvananthapuram, Kerala, 695582, India, Trivandrum",Trivandrum,https://www.oyorooms.com/162962/?rooms=1&guests=2&rooms_config=1-2_0,TVM346,"8.570287251471196,76.86867034150731",OYO Rooms
8,Capital O 84184 Pop Tavern Kazhakkottam,25.141700,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/161573/f556b3cb9425cb35.jpg,999,9,3.9 (27),"Menamkulam, Kazhakkuttam - Menamkulam Road, Kazhakkoottam, Thiruvananthapuram Trivandrum KeralaIndia695585, Trivandrum",Trivandrum,https://www.oyorooms.com/161573/?rooms=1&guests=2&rooms_config=1-2_0,TVM345,"8.560838075233422,76.8585046549992",Capital O
9,SPOT ON 60470 Sreepadmini Ss Ayurmadam,25.160500,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/81135/dda238e876fe8d70.jpg,1029,2,4.0 (393),"Anchalummood.Chanthakkada road,Opposite Anju auditorium, Kollam",Kollam,https://www.oyorooms.com/81135/?rooms=1&guests=2&rooms_config=1-2_0,KOM026,"8.934239,76.602698",SPOT ON


In [77]:
excel_style_df.to_excel("oyo_"+search_data['responseObject'][0]['name']+"_scrape.xlsx",sheet_name='OYO_'+search_data['responseObject'][0]['name']) 

In [78]:
#To Load more Data

In [79]:
pageno = 1
listcount = 20

In [80]:
apiloadmore = 'id,name,city,street,category,geo_location,category,hotel_type,alternate_name,country_name,country_id,short_address,address,hotel_name_without_category,categorywise_images,category_wise_media,category_availability,external_booking_url,status&additional_fields=hotel_badge,category_info,cancellation_policies,best_image,room_pricing,availability,amenities,restrictions,category,captains_info,new_applicable_filters,additional_charge_info,images,hotel_images,collection_filtered_count,popular_location_range,guest_ratings,oyo_wizard,urgency_info,oyo_owner_discount,category_wise_pricing&tax_exclusive_pricing=true&available_room_count[checkin]={cin}&available_room_count[checkout]={cout}&available_room_count[min_count]=1&filters[page]={pgno}&filters[coordinates][latitude]={latitude}&filters[coordinates][longitude]={logitude}&filters[all_room_categories]=true&pre_apply_coupon_switch=true&search_redirection_enabled=true&rooms_config=0,1,0&requested_coupon=&services=meals&source=Web Booking&format_response[batch][count]=20&format_response[batch][offset]={lstcount}&format_response[sort_params][sort_on]=&format_response[sort_params][ascending]=true&locale=en'.format(cin = checkin, cout = checkout, logitude = log, latitude = lat, pgno = pageno, lstcount = listcount)

In [81]:
print('https://www.oyorooms.com/api/pwa/search/hotels?fields=' + urllib.parse.quote(apiloadmore,safe='=&'))

https://www.oyorooms.com/api/pwa/search/hotels?fields=id%2Cname%2Ccity%2Cstreet%2Ccategory%2Cgeo_location%2Ccategory%2Chotel_type%2Calternate_name%2Ccountry_name%2Ccountry_id%2Cshort_address%2Caddress%2Chotel_name_without_category%2Ccategorywise_images%2Ccategory_wise_media%2Ccategory_availability%2Cexternal_booking_url%2Cstatus&additional_fields=hotel_badge%2Ccategory_info%2Ccancellation_policies%2Cbest_image%2Croom_pricing%2Cavailability%2Camenities%2Crestrictions%2Ccategory%2Ccaptains_info%2Cnew_applicable_filters%2Cadditional_charge_info%2Cimages%2Chotel_images%2Ccollection_filtered_count%2Cpopular_location_range%2Cguest_ratings%2Coyo_wizard%2Curgency_info%2Coyo_owner_discount%2Ccategory_wise_pricing&tax_exclusive_pricing=true&available_room_count%5Bcheckin%5D=12%2F03%2F2022&available_room_count%5Bcheckout%5D=13%2F03%2F2022&available_room_count%5Bmin_count%5D=1&filters%5Bpage%5D=1&filters%5Bcoordinates%5D%5Blatitude%5D=8.7378685&filters%5Bcoordinates%5D%5Blongitude%5D=76.7163358999

In [82]:
pageurl = 'https://www.oyorooms.com/api/pwa/search/hotels?fields=' + urllib.parse.quote(apiloadmore,safe='=&')

In [83]:
listing = requests.get(pageurl, headers = headers)

In [84]:
listing_data = json.loads(listing.text)

In [85]:
hotels_list = listing_data['hotels']

In [86]:
listing_df = pd.DataFrame(hotels_list)

In [87]:
listing_df.keys()

Index(['oyo_wizard', 'shortlisted', 'show_original_name', 'urgency_info_list',
       'distance', 'longitude', 'latitude', 'currency_id',
       'selected_category_id', 'country_id', 'id', 'oyo_id', 'street',
       'map_link', 'geo_location', 'category', 'display_category', 'status',
       'alternate_name', 'name', 'address', 'short_address', 'city',
       'country_name', 'hotel_type', 'hotel_name_without_category',
       'best_image', 'currency_symbol', 'currency_code', 'distance_unit',
       'external_booking_url', 'urgency_info', 'ratings', 'pricing_details',
       'amenities', 'category_wise_media', 'category_wise_pricing',
       'room_categories_with_data', 'cancellation_policies', 'available_rooms',
       'hotel_images', 'hotel_badge', 'restrictions', 'pricing_info', 'images',
       'pricing', 'reduced_room_pricing', 'category_wise_display_pricing',
       'fixed_pricing', 'mrcData', 'rating_text'],
      dtype='object')

In [88]:
hotel_df = json_normalize(hotels_list)

In [89]:
hotel_url = 'https://www.oyorooms.com/' + hotel_df['id'] + '/?rooms=1&guests=2&rooms_config=1-2_0'

In [90]:
excel_df = pd.DataFrame()

In [91]:
excel_df['Hotel Name'] = hotel_df['name']
excel_df['Distance'] = hotel_df['distance']
excel_df['Type'] = hotel_df['hotel_type']
excel_df['Image URL'] = hotel_df['best_image']
excel_df['Price'] = json_normalize(hotel_df['mrcData'].apply(lambda x: x[0]))['finalPrice']
excel_df['Available Rooms'] = hotel_df['available_rooms'].apply(lambda x: x[0])
excel_df['Rating'] = hotel_df['ratings.value'].fillna(0).astype(str) + " (" + hotel_df['ratings.count'].fillna(0).astype(int).astype(str) + ")" 
excel_df['Address'] = hotel_df['address']
excel_df['City'] = hotel_df['city']
excel_df['Hotel link'] = 'https://www.oyorooms.com/' + hotel_df['id'] + '/?rooms=1&guests=2&rooms_config=1-2_0'
excel_df['OYO ID'] = hotel_df['oyo_id']
excel_df['Geo Location'] = hotel_df['geo_location']
excel_df['Category'] = hotel_df['category']

In [92]:
excel_df

,Hotel Name,Distance,Type,Image URL,Price,Available Rooms,Rating,Address,City,Hotel link,OYO ID,Geo Location,Category
0,OYO Flagship 10051 Hotel Tyche Stays,31.5514,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,3280,3,4.4 (1696),"Near Aakulam Bridge, Trivandrum",Trivandrum,https://www.oyorooms.com/15598/?rooms=1&guests...,TVM087,"8.5185361,76.8982865",Flagship
1,OYO 5805 Uthradam Towers,32.6332,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,2717,59,4.0 (21),"Kunukuzhy Road, Kunnukuzhy, Trivandrum",Trivandrum,https://www.oyorooms.com/9711/?rooms=1&guests=...,TVM052,"8.511707,76.905375",OYO Rooms
2,Capital O 68343 Platinum Inn,33.0596,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,2235,12,3.9 (195),"NH bypass, Venpalavattom, Anayara, Trivandrum",Trivandrum,https://www.oyorooms.com/93358/?rooms=1&guests...,TVM279,"8.50754017551075,76.9063506089151",Capital O
3,SPOT ON 75237 The New View Hotel,33.9591,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,899,26,3.6 (387),"thoppil lane, Ulloor, Trivandrum",Trivandrum,https://www.oyorooms.com/105641/?rooms=1&guest...,TVM300,"8.51496135021559,76.9273623637855",SPOT ON
4,OYO 81801 Akash villa,33.7820,OYO Home,https://images.oyoroomscdn.com/uploads/hotel_i...,2065,2,4.9 (503),"ittyaveera scan nd genetic centre, Medical Col...",Trivandrum,https://www.oyorooms.com/113776/?rooms=1&guest...,TVM335,"8.523317749041261,76.93377586081624",Home
5,SPOT ON 79508 New View Hotel Inn,33.9613,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,899,40,3.6 (271),"Kumarapuram Market, Kumarapuram Kannamoola Roa...",Trivandrum,https://www.oyorooms.com/110978/?rooms=1&guest...,TVM321,"8.514999,76.927429",SPOT ON
6,SPOT ON 84737 Radwan Tourist Home,34.4008,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,1590,13,3.6 (69),"GW8M 7G4, Near GG Hospital, Murinjapalam, Thir...",Trivandrum,https://www.oyorooms.com/164128/?rooms=1&guest...,TVM348,"8.515397,76.933655",SPOT ON
7,OYO 78601 Poptavern,34.9810,OYO Home,https://images.oyoroomscdn.com/uploads/hotel_i...,1349,3,3.1 (147),"Lekshmi Towers, Trivandrum",Trivandrum,https://www.oyorooms.com/109902/?rooms=1&guest...,TVM310,"8.519127303822543,76.94489998742938",Home
8,CAPITAL O63491 Peroor Inn,35.4325,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,2008,10,3.7 (43),"Thiruvananthapuram - Ponmudi Rd, Ambalamukku.,...",Trivandrum,https://www.oyorooms.com/85550/?rooms=1&guests...,TVM261,"8.53170695727656,76.9619801267982",Capital O
9,OYO 78623 Collection O Hotel Blueway Residency,35.6010,Hotel,https://images.oyoroomscdn.com/uploads/hotel_i...,2356,5,4.4 (318),"Law College Campus Rd, Thekkumood, Barton Hill...",Trivandrum,https://www.oyorooms.com/109931/?rooms=1&guest...,TVM311,"8.508947427673828,76.942602340132",Collection O


In [93]:
excel_style_df = excel_df.style.set_properties(**{'background-color': 'yellow'}, subset=['OYO ID', 'Geo Location'])

In [94]:
excel_style_df

,Hotel Name,Distance,Type,Image URL,Price,Available Rooms,Rating,Address,City,Hotel link,OYO ID,Geo Location,Category
0,OYO Flagship 10051 Hotel Tyche Stays,31.551400,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/15598/2338acf35f5a9448.jpg,3280,3,4.4 (1696),"Near Aakulam Bridge, Trivandrum",Trivandrum,https://www.oyorooms.com/15598/?rooms=1&guests=2&rooms_config=1-2_0,TVM087,"8.5185361,76.8982865",Flagship
1,OYO 5805 Uthradam Towers,32.633200,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/9711/326fcfaadfd26efd.jpg,2717,59,4.0 (21),"Kunukuzhy Road, Kunnukuzhy, Trivandrum",Trivandrum,https://www.oyorooms.com/9711/?rooms=1&guests=2&rooms_config=1-2_0,TVM052,"8.511707,76.905375",OYO Rooms
2,Capital O 68343 Platinum Inn,33.059600,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/93358/59cbc032e92cb722.jpg,2235,12,3.9 (195),"NH bypass, Venpalavattom, Anayara, Trivandrum",Trivandrum,https://www.oyorooms.com/93358/?rooms=1&guests=2&rooms_config=1-2_0,TVM279,"8.50754017551075,76.9063506089151",Capital O
3,SPOT ON 75237 The New View Hotel,33.959100,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/105641/912bf0f3222d42f1.jpg,899,26,3.6 (387),"thoppil lane, Ulloor, Trivandrum",Trivandrum,https://www.oyorooms.com/105641/?rooms=1&guests=2&rooms_config=1-2_0,TVM300,"8.51496135021559,76.9273623637855",SPOT ON
4,OYO 81801 Akash villa,33.782000,OYO Home,https://images.oyoroomscdn.com/uploads/hotel_image/113776/ef012c0e3b13b348.jpg,2065,2,4.9 (503),"ittyaveera scan nd genetic centre, Medical College - Chalakkuzhy Road, Ulloor, Thiruvananthapuram, Kerala, 695001, India, Trivandrum",Trivandrum,https://www.oyorooms.com/113776/?rooms=1&guests=2&rooms_config=1-2_0,TVM335,"8.523317749041261,76.93377586081624",Home
5,SPOT ON 79508 New View Hotel Inn,33.961300,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/110978/54bb4a289dbc6960.jpg,899,40,3.6 (271),"Kumarapuram Market, Kumarapuram Kannamoola Road, Krishna Vihar, Trivandrum",Trivandrum,https://www.oyorooms.com/110978/?rooms=1&guests=2&rooms_config=1-2_0,TVM321,"8.514999,76.927429",SPOT ON
6,SPOT ON 84737 Radwan Tourist Home,34.400800,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/164128/d18eaed5f0435515.jpg,1590,13,3.6 (69),"GW8M 7G4, Near GG Hospital, Murinjapalam, Thiruvananthapuram, Kerala 695033, India, Trivandrum, Trivandrum",Trivandrum,https://www.oyorooms.com/164128/?rooms=1&guests=2&rooms_config=1-2_0,TVM348,"8.515397,76.933655",SPOT ON
7,OYO 78601 Poptavern,34.981000,OYO Home,https://images.oyoroomscdn.com/uploads/hotel_image/109902/897a2844d03eb5c7.JPG,1349,3,3.1 (147),"Lekshmi Towers, Trivandrum",Trivandrum,https://www.oyorooms.com/109902/?rooms=1&guests=2&rooms_config=1-2_0,TVM310,"8.519127303822543,76.94489998742938",Home
8,CAPITAL O63491 Peroor Inn,35.432500,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/85550/ee3526ad416bbf98.jpg,2008,10,3.7 (43),"Thiruvananthapuram - Ponmudi Rd, Ambalamukku., Trivandrum",Trivandrum,https://www.oyorooms.com/85550/?rooms=1&guests=2&rooms_config=1-2_0,TVM261,"8.53170695727656,76.9619801267982",Capital O
9,OYO 78623 Collection O Hotel Blueway Residency,35.601000,Hotel,https://images.oyoroomscdn.com/uploads/hotel_image/109931/e92084cd486ab622.jpg,2356,5,4.4 (318),"Law College Campus Rd, Thekkumood, Barton Hill Colony, Kunnukuzhy, Thiruvananthapuram, Trivandrum",Trivandrum,https://www.oyorooms.com/109931/?rooms=1&guests=2&rooms_config=1-2_0,TVM311,"8.508947427673828,76.942602340132",Collection O


In [97]:
excel_style_df.to_excel("2oyo_"+search_data['responseObject'][0]['name']+"_scrape.xlsx",sheet_name='OYO_'+search_data['responseObject'][0]['name']) 